In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import os

In [ ]:
# --- Step 1: Hyperparameters and Setup ---
# Define key training parameters
BATCH_SIZE = 64
LATENT_DIM = 100  # Dimension of the random noise vector
IMAGE_SIZE = 28 * 28  # 28x28 pixels
NUM_EPOCHS = 50
LEARNING_RATE = 0.0002

In [ ]:

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# Create a directory to save generated images
if not os.path.exists('generated_images'):
    os.makedirs('generated_images)

In [ ]:
# --- Step 2: Define the Generator and Discriminator Networks ---

class Generator(nn.Module):
    """
    The Generator network takes a random noise vector as input and outputs a fake image.
    """
    